In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
dataset = '/content/MyNews - Sheet1.csv'

In [ ]:
import chardet
with open('/content/MyNews - Sheet1.csv', 'rb') as f:
    enc = chardet.detect(f.read())

In [ ]:
data = pd.read_csv(dataset)
print(data.head())

   index                                          Sentences
0      0  အန်ဒရီယာ မာစီ သည် အီတလီ အတွက် စမ်းသပ်မှု တစ်ခု...
1      1  ပေါ်တူဂီ သည် ဘယ်သောအခါမှ စွန့်လွှတ်မှု မရှိခဲ့...
2      2  အီတလီ သည် ပထမပိုင်း ၌ ၁၆-၅ ဖြင့် ဦးဆောင်ခဲ့ သေ...
3      3  သို့သော်လည်း ဘော်တိုလပ်စီ သည် မော်ရို ဘာဂမ်မက်...
4      4  နယူးဇီလန် သည် ရမှတ် ကွာခြားချက် အပေါ် စကော့တလန...


In [ ]:
data.isnull().sum()

index        0
Sentences    4
dtype: int64

In [ ]:
stopwordslist = []
slist = []
with open("/content/stopword (1).txt", encoding = 'utf8') as stopwordsfile:
    stopwords = stopwordsfile.readlines()
    slist.extend(stopwords)
    for w in range(len(slist)):
        temp = slist[w]
        stopwordslist.append(temp.rstrip())

In [ ]:
import re
CleanPattern = re.compile(r'\d+|[၊။!-/:-@[-`{-~\t ]|[A-za-z0-9]')
def clean_sentence(sentence):
     sent = CleanPattern.sub(" ",sentence)
     return sent

def stop_word(sentence):
  new_sentence = []
  for word in sentence.split():
    if word not in stopwordslist:
      new_sentence.append(word)
  return(' '.join(new_sentence))

def process(line):
    line = clean_sentence(line)
    sentence = stop_word(line)
    return sentence

BreakPattern = re.compile(r'(?:(?<!္)([က-ဪဿ၊-၏]|[၀-၉]+|[^က-၏]+)(?![ှျ]?[့္်]))')

def sylbreak(sentence):
    line = BreakPattern.sub(" " + r"\1",sentence)
    return line
       
def process(sentence):
    line = clean_sentence(sentence)
    sentence = stop_word(line)
    return sentence

In [ ]:
data['Sentences'] = data['Sentences'].apply(str)
data['Sentences'].apply(process)
data['Sentences']

0        အန်ဒရီယာ မာစီ သည် အီတလီ အတွက် စမ်းသပ်မှု တစ်ခု...
1        ပေါ်တူဂီ သည် ဘယ်သောအခါမှ စွန့်လွှတ်မှု မရှိခဲ့...
2        အီတလီ သည် ပထမပိုင်း ၌ ၁၆-၅ ဖြင့် ဦးဆောင်ခဲ့ သေ...
3        သို့သော်လည်း ဘော်တိုလပ်စီ သည် မော်ရို ဘာဂမ်မက်...
4        နယူးဇီလန် သည် ရမှတ် ကွာခြားချက် အပေါ် စကော့တလန...
                               ...                        
13882    လက်မှတ်ရေးထိုး မည် ဟု ယုံကြည်ကြောင်း ဝန်ကြီးချ...
13883    လက်နက်ကိုင် များ က ငြင်းပယ် နေ ခြင်း မဟုတ်ဘဲ အ...
13884    အဆင်သင့် မဖြစ်သေး သော အဖွဲ့အစည်းများ သာ ရှိကြေ...
13885                                                  nan
13886                                                  nan
Name: Sentences, Length: 13887, dtype: object

In [ ]:
def build_corpus(data):
  corpus = []
  for col in ['Sentences']:
    for sentence in data[col].iteritems():
      wordlist = sentence[1].split(" ")
      corpus.append(wordlist)
  return corpus

In [ ]:
corpus = build_corpus(data)
corpus[:10]

[['အန်ဒရီယာ',
  'မာစီ',
  'သည်',
  'အီတလီ',
  'အတွက်',
  'စမ်းသပ်မှု',
  'တစ်ခု',
  'အဖြစ်',
  'စတုတ္ထ',
  'မိနစ်',
  'တွင်',
  'အမှတ်ပေးခြင်း',
  'ကို',
  'ဖွင့်လှစ်ပေးခဲ့သည်',
  '။'],
 ['ပေါ်တူဂီ',
  'သည်',
  'ဘယ်သောအခါမှ',
  'စွန့်လွှတ်မှု',
  'မရှိခဲ့',
  'ပဲ',
  'ဒေးဗစ်',
  'ပန်နယ်ဗါ',
  'သည်',
  '၃၃',
  'မိနစ်',
  'တွင်',
  'သူတို့၏',
  'ကစားပွဲ',
  'ရဲ့',
  'အမှတ်များ',
  'ကို',
  'သာ',
  'အထောက်အကူပြု',
  'သည့်',
  'ကြိုးစားမှု',
  'တစ်ခု',
  'မှ',
  'အမှတ်ရခဲ့ပါသည်။'],
 ['အီတလီ',
  'သည်',
  'ပထမပိုင်း',
  '၌',
  '၁၆-၅',
  'ဖြင့်',
  'ဦးဆောင်ခဲ့',
  'သော်လည်း',
  'ပေါ်တူဂီ',
  'မှ',
  'ဒုတိယပိုင်း',
  'တော်တော်များများ',
  'တွင်',
  'ဂိုး',
  'အရေအတွက်',
  'ညီ',
  'စေ',
  'ခဲ့သည်',
  '။'],
 ['သို့သော်လည်း',
  'ဘော်တိုလပ်စီ',
  'သည်',
  'မော်ရို',
  'ဘာဂမ်မက်စကို',
  '၏',
  'ကြိုးစားမှုများ',
  'နှင့်',
  'အန်ဒရီယာ',
  'မာစီ',
  '၏',
  'ဒုတိယဂိုး',
  'နောက်ပိုင်း',
  'အီတလီ',
  'အတွက်',
  'အနိုင်ရရှိ',
  'စေရန်',
  'သူ၏',
  'စတုတ္ထမြောက်',
  'ကစားပွဲ',
  '၏',
  'ပယ်နယ်လ်တီ',
  '

In [ ]:
!pip install gensim
import gensim

In [ ]:
model = gensim.models.Word2Vec(corpus, size= 32, window=3, min_count=3,iter=100)

In [ ]:
model.wv['ငြိမ်းချမ်းရေး']

array([-3.1282380e+00,  3.2111065e+00,  1.7346760e+00, -2.1277218e+00,
       -1.5117459e+00,  1.6338804e+00, -2.8221838e+00,  5.3387256e+00,
       -3.0438690e+00,  2.7966268e+00,  1.1215039e-01, -1.7969534e+00,
       -1.8059758e+00, -4.6619806e-03,  1.5575612e+00, -3.6059609e-01,
       -5.0411744e+00, -5.6467021e-01,  1.4072078e+00, -1.7613953e-01,
        4.9464136e-01, -3.4221020e+00, -2.9030523e+00, -1.6676605e+00,
       -3.2290459e+00, -3.1005747e+00,  2.8049482e-02,  1.5179551e+00,
       -3.4341733e+00,  4.0139165e+00,  1.8632053e-01,  5.3267676e-01],
      dtype=float32)

In [ ]:
model.most_similar('စီးပွားရေး')[:10]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('အခြေအနေ', 0.6170564889907837),
 ('တရုတ်ပြည်သူ့', 0.608558177947998),
 ('ရခိုင်ပြည်နယ်ရဲ့', 0.5919140577316284),
 ('လုပ်ဆောင်ချက်', 0.5907759666442871),
 ('မြန်မာနိုင်ငံတွင်', 0.5867412686347961),
 ('စည်းမျဉ်းများ', 0.5808330774307251),
 ('အားကောင်း', 0.5758386850357056),
 ('အရေးပါသော', 0.5691541433334351),
 ('အဆင့်ဆင့်', 0.5667726993560791),
 ('ဆက်ဆံရေး', 0.5593760013580322)]

In [ ]:
model.save('model.bin')